In [ ]:
import rasterio
import numpy as np
import cv2
import tqdm

fp = r"G:\Mi unidad\Equipo_Agua\Geo\Data\TLALNEPANTLA DE BAZ\Raster\zona_16.tif"
carpeta_salida = r"C:\Users\52551\Documents\Con_lago"
nombre = 'zona_16'
parametro = 1024



img = rasterio.open(fp)
print('Imagen leida.')
array = img.read()
four_images=[array[2],array[1],array[0],array[3]]
stacked_images = np.stack(four_images, axis=-1)
H,W,D=stacked_images.shape
z=0
for i in tqdm.tqdm(range(int(H/parametro))):
    for j in range(int(W/parametro)):
        array1=stacked_images[parametro*i:parametro*(i+1),parametro*j:parametro*(j+1)]
        cv2.imwrite(carpeta_salida + "/" + nombre +'_' +str(i)+"_"+str(j)+".png",array1)
        break

In [1]:
from codigos import particionar_imagen
fp = r"G:\Mi unidad\Equipo_Agua\Geo\Data\TLALNEPANTLA DE BAZ\Raster\zona_16.tif"
path_salida = "G:\Mi unidad\Equipo_Agua\Geo\Data\TLALNEPANTLA DE BAZ\Raster\Fragmentos"
parametro = 1024

particionar_imagen(fp, parametro=parametro, ruta_salida=path_salida, nombre_salida='zona_07')

ModuleNotFoundError: No module named 'cv2'

In [ ]:
def particionar_imagen (fp:str, parametro:int, ruta_salida:str, nombre_salida:str):
    '''
    (Function)
        Esta funciona toma el archivo "fp" y lo particiona en n-elementos (imagenes) para tener fragmentos de la original
        con un zoom siendo cuadradas
    (Paramateres)
        - fp: Ruta de acceso al archivo .tif (Obligatoriamente) 
        - parametro: cantidad de pixeles que tomara para formar un cuadrado (fragmento)
    '''
    img = rasterio.open(fp)
    array = img.read()
    four_images=[array[2],array[1],array[0],array[3]]
    stacked_images = np.stack(four_images, axis=-1)
    H,W,D=stacked_images.shape
    z=0
    for i in tqdm.tqdm(range(int(H/parametro))):
        for j in range(int(W/parametro)):
            array1=stacked_images[parametro*i:parametro*(i+1),parametro*j:parametro*(j+1)]
            cv2.imwrite(ruta_salida+"/"+nombre_salida+str(i)+"_"+str(j)+".png",array1)

In [ ]:

ds = gdal.Open(fp)
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5] 
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3] 